# **Connect with the Drive**

In [ ]:
# This is an essential part to work in Google Colaboratory
from google.colab import drive
drive.mount('/content/drive')

# **To show the Data Structure**

In [ ]:
import xarray as xr

# Load any NetCDF file (example: SST)
ds = xr.open_dataset("/content/drive/My Drive/File_name.nc")
print(ds)

import numpy as np

# Extract latitudes and longitudes
latitudes = ds["latitude"].values  # Check your dataset for actual names, e.g., "lat"
longitudes = ds["longitude"].values  # Sometimes it is "lon"

# Compute spatial resolution (assuming uniform spacing)
lat_res = np.abs(latitudes[1] - latitudes[0])  # Latitude resolution (degrees)
lon_res = np.abs(longitudes[1] - longitudes[0])  # Longitude resolution (degrees)

print(f"Spatial Resolution: {lat_res:.6f}° x {lon_res:.6f}°")

# **Crop the Dataset with Shapefile**

In [ ]:
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
import matplotlib.pyplot as plt

# ---- Step 1: Load the NetCDF File ----
nc_file = "/content/drive/My Drive/File_name.nc"  # Change this to your NetCDF file path
ds = xr.open_dataset(nc_file)

# ---- Step 2: Open the Shapefile ----
shapefile_path = "/content/drive/My Drive/Shapefile_name.shp"  # Change this to your shapefile path
gdf = gpd.read_file(shapefile_path)

# Ensure the shapefile CRS matches NetCDF file CRS
gdf = gdf.to_crs("EPSG:4326")  # Convert to WGS 1984 (latitude/longitude)

# ---- Step 3: Convert NetCDF Dataset to Raster Format & Clip ----
ds = ds.rio.write_crs("EPSG:4326")  # Assign CRS if missing

# Clip the dataset using the shapefile
ds_cropped = ds.rio.clip(gdf.geometry, gdf.crs)

# ---- Step 4: Save the Cropped NetCDF File ----
output_nc_path = "/content/drive/My Drive/File_name_cropped.nc"  # Change to your output path
ds_cropped.to_netcdf(output_nc_path)

print(f"Cropped NetCDF file saved at: {output_nc_path}")

# ---- Step 5: Select a Specific Day to Plot (e.g., First Available Date) ----
selected_date = ds_cropped.time[0].values  # Select the first date
parameter_name_day = ds_cropped["Parameter_name"].sel(time=selected_date)  # Plot Parameter

# ---- Step 6: Plot the Cropped Parameter Data ----
plt.figure(figsize=(8, 6))
chl_day.plot(cmap="coolwarm")  # Change colormap if needed
plt.title(f"Cropped Parameter_name Data for {str(selected_date)[:10]}")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

# **Time-Series of Average High-Low Parameter Value**

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

# Load the NetCDF file (Replace with your actual file path)
file_path = "/content/drive/My Drive/File_name_cropped.nc"
ds = xr.open_dataset(file_path)

# Replace 'Parameter_name' with the actual variable name in your dataset
parameter = ds['Parameter_name']

# Compute the daily mean parameter (averaged over the spatial domain)
parameter_daily_mean = parameter.mean(dim=["latitude", "longitude"])

# Find max and min values
max_value = parameter_daily_mean.max().values
min_value = parameter_daily_mean.min().values

# Find corresponding dates
max_date = parameter_daily_mean.time[parameter_daily_mean.argmax()].values
min_date = parameter_daily_mean.time[parameter_daily_mean.argmin()].values

# === Plot the Time Series with Max & Min Highlighted ===
plt.figure(figsize=(24, 12))
plt.plot(parameter_daily_mean.time, parameter_daily_mean, label="Daily Parameter_name", color="blue")

# Mark the highest and lowest points
plt.scatter(max_date, max_value, color="red", label=f"Max: {max_value:.2f} on {str(max_date)[:10]}", zorder=3)
plt.scatter(min_date, min_value, color="green", label=f"Min: {min_value:.2f} on {str(min_date)[:10]}", zorder=3)

plt.xlabel("Time")
plt.ylabel("Parameter (Original Unit)")
plt.title("Daily Parameter (2021-2024)")
plt.legend()
plt.grid()
plt.show()

# **Map of Parameter**

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Load the NetCDF file (Replace with your file path)
file_path = "/content/drive/My Drive/File_name_cropped.nc"
ds = xr.open_dataset(file_path)

# Replace 'Parameter_name' with the actual variable name in your dataset
parameter = ds['Parameter_name']

# Plot parameter as a Map for a Specific Day ===
selected_date = "2023-08-16"  # Change this to your desired date
parameter_day = parameter.sel(time=selected_date, method="nearest")

plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
parameter_day.plot.pcolormesh(ax=ax, cmap="viridis", transform=ccrs.PlateCarree())

ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=":")
ax.gridlines(draw_labels=True)

plt.title(f"Parameter on {selected_date}")
plt.show()

# **Checking the High-Low value**

In [ ]:
import xarray as xr

# Load the original NetCDF file
file_path = "/content/drive/My Drive/File_name_cropped.nc"  # Update path if needed
ds = xr.open_dataset(file_path)

# Extract min and max values
parameter_min, parameter_max = ds["Parameter_name"].min().item(), ds["Parameter_name"].max().item()

# Print results in correct units
print(f"🌡️ parameter (Original Unit): {parameter_min:.2f}Original Unit to {parameter_max:.2f}Original Unit")

# **Checking Parameter Metadata**

In [ ]:
import xarray as xr

# Load the original NetCDF file
file_path = "/content/drive/My Drive/File_name_cropped.nc"  # Update with actual path
ds = xr.open_dataset(file_path)

# Display dataset details
print(ds)

# Print attributes of each variable (to check units)
for var in ds.variables:
    print(f"\n🔹 Parameter: {var}")
    print(ds[var].attrs)  # Prints metadata, including units if available


# **Normalization of Dataset (0-255 pixel)**

In [ ]:
import xarray as xr
from sklearn.preprocessing import MinMaxScaler
import os

# Load original NetCDF file
file_path = "/content/drive/My Drive/File_name_cropped.nc"  # Change this to your actual file path
ds = xr.open_dataset(file_path)

# Define output folder
output_folder = "/content/drive/My Drive/Normalized_NetCDF"
os.makedirs(output_folder, exist_ok=True)  # Create folder if it doesn’t exist

# Define parameters to scale
parameters = ['Parameter_name']

# Apply Min-Max Scaling (0 to 255) for CNN
ds_cnn = ds.copy()
scaler_0_255 = MinMaxScaler(feature_range=(0, 255))
for param in parameters:
    data = ds_cnn[param].values.reshape(-1, 1)  # Reshape for sklearn
    scaled_data = scaler_0_255.fit_transform(data).reshape(ds_cnn[param].shape)
    ds_cnn[param] = (ds_cnn[param].dims, scaled_data)

# Save the CNN-normalized dataset
cnn_output_path = os.path.join(output_folder, "normalized_0_255.nc")
ds_cnn.to_netcdf(cnn_output_path)
print(f"CNN Normalization (0-255) applied and saved as {cnn_output_path}")

print("\n✅ Normalization completed successfully! Files saved in 'Normalized NetCDF' folder.")

# **EDA of Normalize Dataset**

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns

# Load the normalized dataset (0-255 scaled file for CNN)
file_path = "/content/drive/My Drive/normalized_0_255.nc"  # Update the path if needed
ds = xr.open_dataset(file_path)

# Define parameters for EDA
parameter = ['Parameter_name']

# 📌 **1. Dataset Overview**
print("\n🔹 Dataset Structure:\n", ds)
print("\n🔹 Dataset Statistics:\n", ds.to_dataframe().describe())

# 📌 **2. Check Missing Values**
print("\n🔹 Missing Values:\n", ds.to_dataframe().isnull().sum())

# 📌 **3. Plot Time Series Trends (Global Mean Values)**
plt.figure(figsize=(12, 5))
for param in parameters:
    ds[param].mean(dim=['latitude', 'longitude']).plot(label=param)
plt.legend()
plt.title("📈 Daily Trends of Parameter")
plt.xlabel("Time")
plt.ylabel("Pixel Intensity (0-255)")
plt.show()

# 📌 **4. Plot Histograms & Distributions**
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for i, param in enumerate(parameters):
    sns.histplot(ds[param].values.flatten(), bins=50, kde=True, ax=axes[i])
    axes[i].set_title(f"Distribution of {param}")
plt.tight_layout()
plt.show()

# 📌 **6. Generate Spatial Maps (First Time Step)**
for param in parameter:
    ds[param].isel(time=0).plot(cmap="viridis", figsize=(8, 6))
    plt.title(f"🗺️ Spatial Distribution of {param} (First Time Step)")
    plt.show()

print("\n✅ CNN-Based EDA Completed Successfully!")

# **Split daily images from the original Combined Dataset**

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image  # For resizing images

# Define your Google Drive base path
base_path = "/content/drive/My Drive/Images"

# Define output folders for images inside Google Drive
output_dirs = {
    "Parameter_name": os.path.join(base_path, "Parameter")}

# Create directories if they don’t exist
for path in output_dirs.values():
    os.makedirs(path, exist_ok=True)

# Load your normalized NetCDF file
file_path = "/content/drive/My Drive/normalized_0_255.nc"  # Update this path
ds = xr.open_dataset(file_path)

# Define colormaps for visualization
colormaps = {
    "{Parameter}": "choose colorcode"}   # Choose color for Parameter

# Define target image resolution (change if needed)
target_size = (512, 512)  # Resize images to 512x512 pixels

# Convert each parameter to color images
for param in ["Parameter_name"]:
    data = ds[param]  # Extract parameter data

    # Get the corresponding colormap key
    colormap_key = {
        "Parameter_name": "Parameter"}.get(param, param)  # Default to param if not found

    for i, timestamp in enumerate(data.time):
        img_data = data.sel(time=timestamp).values  # Extract 2D array
        img_data = img_data.squeeze()
        img_data = np.nan_to_num(img_data, nan=0)  # Convert NaN (land) to 0 (black)

        # Fix flipping and rotation issues
        img_data = np.flipud(img_data)  # Flip vertically
        #img_data = np.fliplr(img_data)  # Flip horizontally

        # Create and save temporary image
        plt.figure(figsize=(4, 4))
        plt.axis("off")
        # Use colormap_key instead of param directly
        plt.imshow(img_data, cmap=colormaps[colormap_key])  # Apply colormap
        #plt.colorbar(label=param)

        # Temporary image path
        temp_filename = f"{output_dirs[param]}/temp_{i:04d}.png"
        plt.savefig(temp_filename, bbox_inches="tight", pad_inches=0)
        plt.close()

        # Resize image using PIL
        img = Image.open(temp_filename)
        img_resized = img.resize(target_size, Image.Resampling.LANCZOS)

        # Final resized image path
        final_filename = f"{output_dirs[param]}/{param}_{i:04d}.png"
        img_resized.save(final_filename)

        # Remove temporary image
        os.remove(temp_filename)

    print(f"✅ Saved {len(data.time)} resized images for {param} in Google Drive.")

# **Dataset Labeling**

In [ ]:
import os
import cv2
import numpy as np

def convert_threshold_to_pixel(original_value, param_min, param_max):
    """
    Convert an original threshold value to its corresponding pixel value
    for an 8-bit (0-255) normalized image.
    """
    return (original_value - param_min) / (param_max - param_min) * 255

# ----- Define thresholds and ranges for each parameter -----

# Example values for Parameter (in Original Unit)
parameter_min = give value        # Original minimum Parameter
parameter_max = give value        # Original maximum Parameter
parameter_topic_lower_orig = give value   # topic lower threshold in Original Unit for Parameters
parameter_topic_upper_orig = give value   # topic upper threshold in Original Unit for Parameters

# Convert each parameter's thresholds to pixel values
Parameter_topic_lower_px = convert_threshold_to_pixel(parameter_topic_lower_orig, parameter_min, parameter_max)
Parameter_topic_upper_px = convert_threshold_to_pixel(parameter_topic_upper_orig, parameter_min, parameter_max)

print("Parameter topic pixel range: {:.2f} - {:.2f}".format(parameter_topic_lower_px, parameter_topic_upper_px))

# ----- Define input and output directories for parameter -----
input_dirs = {
    "parameter": "/content/drive/My Drive/Images/Parameter"}      # Folder with Parameter PNG files

output_dirs = {
    "parameter": "/content/drive/My Drive/Images/Parameter_Labeled"}   # Output folder for labeled Parameter images

# Create output directories if they don't exist
for path in output_dirs.values():
    os.makedirs(path, exist_ok=True)

def process_images_for_parameter(param_key, lower_px, upper_px):
    """
    Process all images for a given parameter.
    For each image:
      - Read the normalized (0-255) grayscale image.
      - Create a binary mask: set pixel to 1 (Topic) if its value is between lower_px and upper_px, else 0.
      - Save the binary mask as a PNG (multiplying by 255 for visualization).
    """
    input_dir = input_dirs[param_key]
    output_dir = output_dirs[param_key]
    image_files = sorted([f for f in os.listdir(input_dir) if f.lower().endswith('.png')])

    for i, fname in enumerate(image_files):
        image_path = os.path.join(input_dir, fname)
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Create binary mask: 1 for HAB area, 0 for non-HAB
        binary_mask = np.where((img >= lower_px) & (img <= upper_px), 1, 0).astype(np.uint8)

        # Save the binary mask as PNG; multiply by 255 for display purposes (0 remains 0, 1 becomes 255)
        output_path = os.path.join(output_dir, fname)
        cv2.imwrite(output_path, binary_mask * 255)

        if (i+1) % 100 == 0 or (i+1) == len(image_files):
            print(f"[{param_key.upper()}] Processed {i+1}/{len(image_files)} images.")

    print(f"Finished processing {param_key.upper()} images. Labeled images saved in {output_dir}")

# Process images for each parameter using its respective threshold pixel values
process_images_for_parameter("parameter", parameter_bloom_lower_px, parameter_bloom_upper_px)

# **Dataset spliting into test, train & val**

In [ ]:
import os
import shutil
import random

# Define paths
base_dir = "/content/drive/My Drive/Images"  # Original images
output_dir = "/content/drive/My Drive/Images/Split_Images"  # Destination for split images
os.makedirs(output_dir, exist_ok=True)

# Categories: SST, SSS, Chl-a
categories = ["Parameter"]
split_ratios = {"train": 0.7, "val": 0.15, "test": 0.15}

# Function to split and copy images
def split_and_copy(category):
    print(f"Processing: {category}")

    # Define input directories
    rgb_dir = os.path.join(base_dir, category)
    label_dir = os.path.join(base_dir, f"{category}_Labeled")

    # Define output directories
    for split in ["train", "val", "test"]:
        os.makedirs(os.path.join(output_dir, category, split), exist_ok=True)
        os.makedirs(os.path.join(output_dir, f"{category}_Labeled", split), exist_ok=True)

    # Get list of image files (sorted to avoid shuffling mismatched pairs)
    image_files = sorted([f for f in os.listdir(rgb_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])
    random.seed(42)
    random.shuffle(image_files)

    # Determine split sizes
    total = len(image_files)
    train_end = int(total * split_ratios["train"])
    val_end = train_end + int(total * split_ratios["val"])

    # Perform splitting
    split_data = {
        "train": image_files[:train_end],
        "val": image_files[train_end:val_end],
        "test": image_files[val_end:]
    }

    # Copy images to respective folders
    for split, files in split_data.items():
        for file in files:
            shutil.copy(os.path.join(rgb_dir, file), os.path.join(output_dir, category, split, file))
            shutil.copy(os.path.join(label_dir, file), os.path.join(output_dir, f"{category}_Labeled", split, file))

    print(f"✅ {category} and {category}_Labeled split successfully!")

# Run the splitting process
for cat in categories:
    split_and_copy(cat)

print("🎯 All datasets split successfully!")

# **CNN Run**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import os

# Define dataset paths for SST
IMG_SIZE = (512, 512)  # Input image size
BATCH_SIZE = 4  # Adjust based on GPU memory
train_input_dir = "/content/drive/My Drive/Images/Split_Images/Parameter/train"  # Colorful images (SST)
train_mask_dir = "/content/drive/My Drive/Images/Split_Images/Parameter_Labeled/train"  # Labeled images (HAB: 1, non-HAB: 0)
val_input_dir = "/content/drive/My Drive/Images/Split_Images/Parameter/val"
val_mask_dir = "/content/drive/My Drive/Images/Split_Images/Parameter_Labeled/val"
test_input_dir = "/content/drive/My Drive/Images/Split_Images/Parameter/test"
test_mask_dir = "/content/drive/My Drive/Images/Split_Images/Parameter_Labeled/test"

# Function to load and preprocess images & masks
def process_path(image_path, mask_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, IMG_SIZE) / 255.0

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_jpeg(mask, channels=1)
    mask = tf.image.resize(mask, IMG_SIZE) / 255.0

    return image, mask

# Create dataset loaders
def create_dataset(image_dir, mask_dir):
    image_paths = sorted([os.path.join(image_dir, fname) for fname in os.listdir(image_dir)])
    mask_paths = sorted([os.path.join(mask_dir, fname) for fname in os.listdir(mask_dir)])

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))
    dataset = dataset.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return dataset

train_ds = create_dataset(train_input_dir, train_mask_dir)
val_ds = create_dataset(val_input_dir, val_mask_dir)
test_ds = create_dataset(test_input_dir, test_mask_dir)

# U-Net Model Definition
def unet_model(input_size=(512, 512, 3)):
    inputs = layers.Input(input_size)

    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(conv5)

    up6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv5)
    up6 = layers.concatenate([up6, conv4])
    conv6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(up6)
    conv6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)

    up7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    up7 = layers.concatenate([up7, conv3])
    conv7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(up7)
    conv7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)

    up8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    up8 = layers.concatenate([up8, conv2])
    conv8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(up8)
    conv8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)

    up9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    up9 = layers.concatenate([up9, conv1])
    conv9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(up9)
    conv9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(conv9)
    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

# Train Model
model = unet_model()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

EPOCHS = 50
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    keras.callbacks.ModelCheckpoint("/content/drive/My Drive/Topic_U-Net_Best_Model_chl.keras", save_best_only=True)
]

history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=callbacks)

# Evaluate Model
best_model = keras.models.load_model("/content/drive/My Drive/Topic_U-Net_Best_Model_chl.keras")
best_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

test_loss, test_acc = best_model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc}")

# **X_Y_Test_Parameter**

In [ ]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from glob import glob
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# === Set Directories ===
image_dir = "/content/drive/My Drive/Split_Images/Parameter/test"
mask_dir = "/content/drive/My Drive/Split_Images/Parameter_Labeled/test"
save_path = "/content/drive/My Drive/X_Y_Test_Parameter/"
os.makedirs(save_path, exist_ok=True)

# === Load All Image and Mask Paths ===
image_paths = sorted(glob(os.path.join(image_dir, "*.png")))  # Change extension if needed
mask_paths = sorted(glob(os.path.join(mask_dir, "*.png")))

# === Filter Only Matching Pairs ===
matched_image_paths = []
matched_mask_paths = []
for img_path, mask_path in zip(image_paths, mask_paths):
    if os.path.exists(img_path) and os.path.exists(mask_path):
        matched_image_paths.append(img_path)
        matched_mask_paths.append(mask_path)

print(f"Found {len(matched_image_paths)} matched image-mask pairs.")

# === Load and Normalize Images ===
X = np.array([img_to_array(load_img(p, target_size=(512, 512))) / 255.0 for p in matched_image_paths], dtype=np.float32)
Y = np.array([img_to_array(load_img(p, target_size=(512, 512), color_mode="grayscale")) / 255.0 for p in matched_mask_paths], dtype=np.float32)

# === Ensure Binary Masks ===
Y = (Y > 0.5).astype(np.uint8)

# === Check Dataset Size ===
if len(X) == 0 or len(Y) == 0:
    raise ValueError("❌ No data found. Please check that your image and mask directories are correct and contain valid .png files.")

# === Split Dataset ===
X_trainval, X_test, Y_trainval, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# === Save Test Set ===
np.save(os.path.join(save_path, "X_test_parameter.npy"), X_test)
np.save(os.path.join(save_path, "Y_test_parameter.npy"), Y_test)

# === Print Summary ===
print("✅ Dataset loaded, preprocessed, split, and test set saved.")
print(f"X_test shape: {X_test.shape}")
print(f"Y_test shape: {Y_test.shape}")

# **Evaluation of Model**

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve, accuracy_score, cohen_kappa_score, precision_score, recall_score

# === Step 1: Select Directory to Save Outputs ===
output_dir = "/content/drive/My Drive/Topic_Results"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# === Step 2: Load Best Chl-a Model ===
model_path = "/content/drive/My Drive/Topic_U-Net_Best_Model_chl.keras"
model = tf.keras.models.load_model(model_path)

# === Step 3: Load Test Data ===
X_test = np.load("/content/drive/My Drive/X_Y_Test_Parameter/X_test_parameter.npy")
Y_test = np.load("/content/drive/My Drive/X_Y_Test_Parameter/Y_test_parameter.npy")

# === Step 4: Generate Predictions ===
Y_pred = model.predict(X_test, batch_size=4)
Y_pred_bin = (Y_pred > 0.5).astype(np.uint8)

# === Step 5: Compute Performance Metrics ===
def compute_metrics(y_true, y_pred):
    cm = confusion_matrix(y_true.flatten(), y_pred.flatten())
    tn, fp, fn, tp = cm.ravel()
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    iou = tp / (tp + fp + fn) if (tp + fp + fn) > 0 else 0
    dice = (2 * tp) / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    kappa = cohen_kappa_score(y_true.flatten(), y_pred.flatten())
    return {"Accuracy": accuracy, "Precision": precision, "Recall": recall, "IoU": iou, "Dice": dice, "F1-score": f1_score, "Kappa": kappa}

metrics = compute_metrics(Y_test, Y_pred_bin)
metrics_json_path = os.path.join(output_dir, "Topic_model_metrics.json")
with open(metrics_json_path, "w") as f:
    json.dump(metrics, f, indent=4)
metrics_csv_path = os.path.join(output_dir, "Topic_model_metrics.csv")
df_metrics = pd.DataFrame([metrics])
df_metrics.to_csv(metrics_csv_path, index=False)
print("✅ Model Performance Metrics (JSON & CSV saved):")
print(json.dumps(metrics, indent=4))

# === Step 6: Confusion Matrix ===
cm = confusion_matrix(Y_test.flatten(), Y_pred_bin.flatten())
plt.figure(figsize=(24, 24))
sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", xticklabels=["Non-Topic", "Topic"], yticklabels=["Non-Topic", "Topic"], annot_kws={"size": 48}, cbar_kws={"label": "Count"})
plt.xlabel("Predicted", fontsize=40)
plt.ylabel("Actual", fontsize=40)
plt.title("Confusion Matrix - Topic Model", fontsize=44)
plt.xticks(fontsize=36)
plt.yticks(fontsize=36)
cm_path = os.path.join(output_dir, "Topic_confusion_matrix.png")
plt.savefig(cm_path, dpi=300)
plt.show()

# === Step 7: ROC Curve & AUC Score ===
fpr, tpr, _ = roc_curve(Y_test.flatten(), Y_pred.flatten())
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(24, 24))
plt.plot(fpr, tpr, color="blue", lw=4, label=f"AUC = {roc_auc:.4f}")
plt.plot([0, 1], [0, 1], color="grey", linestyle="--")
plt.xlabel("False Positive Rate", fontsize=40)
plt.ylabel("True Positive Rate", fontsize=40)
plt.title("ROC Curve - Topic Model", fontsize=44)
plt.legend(loc="lower right", fontsize=36)
plt.xticks(fontsize=36)
plt.yticks(fontsize=36)
roc_path = os.path.join(output_dir, "Topic_roc_curve.png")
plt.savefig(roc_path, dpi=300)
plt.show()

# === Step 8: Precision-Recall Curve ===
precision, recall, _ = precision_recall_curve(Y_test.flatten(), Y_pred.flatten())
plt.figure(figsize=(24, 24))
plt.plot(recall, precision, color="green", lw=4)
plt.xlabel("Recall", fontsize=40)
plt.ylabel("Precision", fontsize=40)
plt.title("Precision-Recall Curve - Topic Model", fontsize=44)
plt.xticks(fontsize=36)
plt.yticks(fontsize=36)
pr_path = os.path.join(output_dir, "Topic_precision_recall_curve.png")
plt.savefig(pr_path, dpi=300)
plt.show()

# === Step 10: Visualize Predicted HAB Map ===
num_samples = 5  # Number of test samples to visualize
fig, axes = plt.subplots(num_samples, 3, figsize=(40, num_samples * 12))  # Increased figure size

for i in range(num_samples):
    axes[i, 0].imshow(X_test[i])  # Original Chl-a image
    axes[i, 0].set_title("Parameter Image", fontsize=36)

    axes[i, 1].imshow(Y_test[i].squeeze(), cmap="gray")  # Ground Truth
    axes[i, 1].set_title("Ground Truth (Topic)", fontsize=36)

    axes[i, 2].imshow(Y_pred_bin[i].squeeze(), cmap="gray")  # Predicted HAB
    axes[i, 2].set_title("Predicted Topic", fontsize=36)

    for ax in axes[i]:
        ax.axis("off")

plt.tight_layout()

# Save Final HAB Maps Image
hab_maps_path = os.path.join(output_dir, "Parameter_Topic_maps.png")
plt.savefig(hab_maps_path, dpi=300)
plt.show()

# === Step 9: F1-Score vs. Threshold Curve ===
thresholds = np.linspace(0, 1, 100)
f1_scores = []

# Iterate over each threshold to calculate the F1 score
for threshold in thresholds:
    Y_pred_bin = (Y_pred > threshold).astype(np.uint8)
    precision = precision_score(Y_test.flatten(), Y_pred_bin.flatten(), zero_division=0)
    recall = recall_score(Y_test.flatten(), Y_pred_bin.flatten(), zero_division=0)
    if precision + recall > 0:
        f1 = 2 * (precision * recall) / (precision + recall)
    else:
        f1 = 0
    f1_scores.append(f1)

plt.figure(figsize=(24, 24))
plt.plot(thresholds, f1_scores, color="red", lw=4)
plt.xlabel("Threshold", fontsize=40)
plt.ylabel("F1 Score", fontsize=40)s
plt.title("F1 Score vs. Threshold", fontsize=44)
f1_path = os.path.join(output_dir, "Parameter_f1_threshold_curve.png")
plt.savefig(f1_path, dpi=300)
plt.show()

print("✅ All results saved successfully!")

# **Lat-Long Save**

In [ ]:
from netCDF4 import Dataset
import numpy as np

# Define the path to your combined NetCDF file
nc_file_path = "/content/drive/My Drive/File_name_cropped.nc"

# Open NetCDF file
nc_data = Dataset(nc_file_path, mode='r')

# Check available variables
print(nc_data.variables.keys())  # This will show lat/lon variable names

# Extract latitude and longitude
latitudes = nc_data.variables['latitude'][:]  # Change to the correct variable name
longitudes = nc_data.variables['longitude'][:]  # Change to the correct variable name

# Convert MaskedArrays to regular NumPy arrays
latitudes = latitudes.filled(np.nan)  # Replace masked values with NaN
longitudes = longitudes.filled(np.nan)  # Replace masked values with NaN

# Save as NumPy files for later use
np.save('/content/drive/My Drive/Split_Images/latitude.npy', latitudes)
np.save('/content/drive/My Drive/Split_Images/longitude.npy', longitudes)

print("✅ Latitude and Longitude extracted and saved!")

# **Grad-CAM Map & Heatmap**

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
import os
import xarray as xr
from scipy.interpolate import griddata

# Define your output directory
output_dir = "/content/drive/My Drive/Topic_Results/"
os.makedirs(output_dir, exist_ok=True)

# Load latitude & longitude from NetCDF
nc_file = '/content/drive/My Drive/File_name_cropped.nc'  # Change to your actual file
with xr.open_dataset(nc_file) as ds:
    latitudes = ds['latitude'].values
    longitudes = ds['longitude'].values

# Create a meshgrid for lat/lon
grid_lon, grid_lat = np.meshgrid(longitudes, latitudes)

# Load test images
X_test_chla = np.load('/content/drive/My Drive/Split_Images/X_Y_Test_Parameter/X_test_parameter.npy')

# Load the best Topic model (for Parameter)
model = tf.keras.models.load_model('/content/drive/My Drive/Topic_U-Net_Best_Model_chl.keras')

# Select a test image
img_index = 0  # Change this index if needed
input_image = np.expand_dims(X_test_parameter[img_index], axis=0)

# Get model's last convolutional layer
last_conv_layer = model.get_layer('conv2d_18')  # Change if needed

# Grad-CAM Heatmap
grad_model = tf.keras.models.Model([model.inputs], [last_conv_layer.output, model.output])
with tf.GradientTape() as tape:
    conv_output, predictions = grad_model(input_image)
    loss = predictions[:, :, :, 0]  # Target class (HAB)

grads = tape.gradient(loss, conv_output)
weights = tf.reduce_mean(grads, axis=(0, 1, 2))
cam_output = np.dot(conv_output[0], weights[..., np.newaxis])
cam_output = np.squeeze(cam_output)

# Normalize & Resize Heatmap
cam_output = np.maximum(cam_output, 0)
cam_output /= np.max(cam_output)
heatmap = cv2.resize(cam_output, (X_test_chla.shape[1], X_test_chla.shape[2]))

# Interpolate to match lat/lon grid
lat_2d, lon_2d = np.meshgrid(np.linspace(latitudes.min(), latitudes.max(), heatmap.shape[0]),
                             np.linspace(longitudes.min(), longitudes.max(), heatmap.shape[1]),
                             indexing='ij')
heatmap_resized = griddata((lon_2d.ravel(), lat_2d.ravel()), heatmap.ravel(), (grid_lon, grid_lat), method='cubic')

# Rotate heatmap by 180 degrees
heatmap_resized = np.rot90(heatmap_resized, 2)

# Apply mirror flip
heatmap_resized = np.fliplr(heatmap_resized)

# Plot Grad-CAM Heatmap
plt.figure(figsize=(72, 54))
plt.contourf(grid_lon, grid_lat, heatmap_resized, cmap='jet', alpha=0.75)
cbar = plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label("Model Focus Intensity", fontsize = 50)
cbar.ax.tick_params(labelsize=40)  # Increase size of color bar numbers
plt.title("Grad-CAM Heatmap of Topic", fontsize=60)
plt.xlabel("Longitude", fontsize=50)
plt.ylabel("Latitude", fontsize=50)
plt.xticks(fontsize=50)
plt.yticks(fontsize=50)
plt.savefig(os.path.join(output_dir, 'GradCAM_Heatmap_Topic.png'))
plt.show()

# ------------------------- #
# ✅ HAB Probability Heatmap
# ------------------------- #

# Get model predictions
preds_chla = model.predict(input_image)
probabilities = preds_chla[0, :, :, 0]

# Interpolate probability heatmap to lat/lon
detected_prob_resized = griddata((lon_2d.ravel(), lat_2d.ravel()), probabilities.ravel(), (grid_lon, grid_lat), method='cubic')

# Rotate probability heatmap by 180 degrees
#detected_prob_resized = np.rot90(detected_prob_resized, 2)

# Apply mirror flip
detected_prob_resized = np.fliplr(detected_prob_resized)

# Fix the flipped heatmap orientation
detected_prob_resized = np.flipud(np.fliplr(detected_prob_resized))

# Plot Probability Heatmap with customized latitude and longitude
plt.figure(figsize=(72, 54))

# Plot heatmap using imshow to create mappable object for colorbar
im = plt.imshow(detected_prob_resized, cmap='jet', alpha=0.75)

# Add colorbar
cbar = plt.colorbar(im, fraction=0.046, pad=0.04)
cbar.set_label("Topic Probability", fontsize=50)
cbar.ax.tick_params(labelsize=40)  # Increase size of color bar numbers

plt.title("Topic Probability Heatmap", fontsize=60)
plt.xlabel("Longitude", fontsize=50)
plt.ylabel("Latitude", fontsize=50)

# Set proper latitude and longitude ticks
plt.xticks(ticks=np.linspace(0, detected_prob_resized.shape[1] - 1, num=5), labels=np.round(np.linspace(longitudes.min(), longitudes.max(), num=5), 2), fontsize=50)
plt.yticks(ticks=np.linspace(0, detected_prob_resized.shape[0] - 1, num=5), labels=np.round(np.linspace(latitudes.min(), latitudes.max(), num=5), 2), fontsize=50)

# Invert the y-axis to fix the latitude orientation
plt.gca().invert_yaxis()

plt.savefig(os.path.join(output_dir, 'Topic_Probability_Heatmap.png'))
plt.show()

print(f"🔥 Heatmaps saved in: {output_dir}")

# **Map with Shapefile for better visualization**

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
import os
import xarray as xr
from scipy.interpolate import griddata
import geopandas as gpd

# Define your output directory
output_dir = "/content/drive/My Drive/Topic_Results/"
os.makedirs(output_dir, exist_ok=True)

# Load the shapefile (change path accordingly)
shapefile_path = "/content/drive/My Drive/Shapefile.shp"  # Update with your actual path
gdf = gpd.read_file(shapefile_path)

# Ensure shapefile CRS matches NetCDF coordinates
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)  # Assuming lat/lon in degrees

gdf = gdf.to_crs(epsg=4326)  # Convert to WGS84 (EPSG:4326) if needed

# Ensure shapefile CRS matches NetCDF coordinates
if gdf1.crs is None:
    gdf1.set_crs(epsg=4326, inplace=True)  # Assuming lat/lon in degrees

gdf1 = gdf1.to_crs(epsg=4326)  # Convert to WGS84 (EPSG:4326) if needed

# Load latitude & longitude from NetCDF
nc_file = '/content/drive/My Drive/File_name_cropped.nc'  # Change to your actual file
with xr.open_dataset(nc_file) as ds:
    latitudes = ds['latitude'].values
    longitudes = ds['longitude'].values

# Create a meshgrid for lat/lon
grid_lon, grid_lat = np.meshgrid(longitudes, latitudes)

# Load test images
X_test_chla = np.load('/content/drive/My Drive/Split_Images/X_Y_Test_Parameter/X_test_parameter.npy')

# Load the best Topic model
model = tf.keras.models.load_model('/content/drive/My Drive/Topic_U-Net_Best_Model_chl.keras')

# Select a test image
img_index = 0  # Change this index if needed
input_image = np.expand_dims(X_test_parameter[img_index], axis=0)

# Get model's last convolutional layer
last_conv_layer = model.get_layer('conv2d_18')  # Change if needed

# Grad-CAM Heatmap
grad_model = tf.keras.models.Model([model.inputs], [last_conv_layer.output, model.output])
with tf.GradientTape() as tape:
    conv_output, predictions = grad_model(input_image)
    loss = predictions[:, :, :, 0]  # Target class (HAB)

grads = tape.gradient(loss, conv_output)
weights = tf.reduce_mean(grads, axis=(0, 1, 2))
cam_output = np.dot(conv_output[0], weights[..., np.newaxis])
cam_output = np.squeeze(cam_output)

# Normalize & Resize Heatmap
cam_output = np.maximum(cam_output, 0)
cam_output /= np.max(cam_output)
heatmap = cv2.resize(cam_output, (X_test_chla.shape[1], X_test_chla.shape[2]))

# Interpolate to match lat/lon grid
lat_2d, lon_2d = np.meshgrid(np.linspace(latitudes.min(), latitudes.max(), heatmap.shape[0]),
                             np.linspace(longitudes.min(), longitudes.max(), heatmap.shape[1]),
                             indexing='ij')
heatmap_resized = griddata((lon_2d.ravel(), lat_2d.ravel()), heatmap.ravel(), (grid_lon, grid_lat), method='cubic')

# Rotate heatmap by 180 degrees
heatmap_resized = np.rot90(heatmap_resized, 2)

# Apply mirror flip
heatmap_resized = np.fliplr(heatmap_resized)

# Plot Grad-CAM Heatmap
plt.figure(figsize=(72, 54))
plt.contourf(grid_lon, grid_lat, heatmap_resized, cmap='jet', alpha=0.75)

# Overlay shapefile
gdf1.plot(ax=plt.gca(), facecolor="peru", edgecolor="black", linewidth=3)    # Change color and linewidth if needed

# Overlay shapefile
gdf.plot(ax=plt.gca(), facecolor="burlywood", edgecolor="black", linewidth=3)    # Change color and linewidth if needed

# Overlay shapefile
#gdf.boundary.plot(ax=plt.gca(), color="Black", edgecolor="Black", linewidth=5)  # Change color and linewidth if needed

cbar = plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label("Model Focus Intensity", fontsize = 50)
cbar.ax.tick_params(labelsize=40)  # Increase size of color bar numbers
plt.title("Grad-CAM Heatmap of Topic", fontsize=60)
plt.xlabel("Longitude (Degrees_East)", fontsize=50)
plt.ylabel("Latitude (Degrees_North)", fontsize=50)
plt.xticks(fontsize=50)
plt.yticks(fontsize=50)
plt.savefig(os.path.join(output_dir, 'GradCAM_Heatmap_Topic_Shp.png'))
plt.show()

print(f"🔥 Heatmap saved in: {output_dir}")